## Best Model 제작을 위한 실험 프로세스 

어떤 실험이 가능한가? 
- 학습 실험 과정 
    - 데이터를 변경하면서 모델 성능 확인 
    - Step 의 코드를 업데이트 하면서 학습 결과 확인
    - Step 의 파라미터를 변경하면서 모델 성능 확인
    - 특정 Step 만 집중적으로 재학습 
</br></br>
 
- 추론 실험 과정    
    - 이전 학습 실험의 모델로 실험 가능  
    - 고정된 모델로 데이터를 변경하면서 추론 결과 확인
    - 고정된 모델로 파라미터를 변경하면서 추론 결과 확인     

</br>

**추론 결과가 좋은 모델을 찾는 것을 목표로 한다. !!**

### Step0. 사전 준비물: 

##### 준비물1: 
```bash
    conda activate {ENV-NAME}           ## python main.py 실행 용 가상환경
    pip install ipykernel        
    python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}
```

##### 준비물2: 
Asset 에 self.asset.save_summary() (ALO API) 를 사용하여 학습 및 추론 결과를 저장합니다.    
- train/inference_pipeline 에서 선언되는 경우 에 따라 train/inference history 에 따로 저장

===========  
```python 
# {solution_name}/assets/output/asset_output.py
summary = {}
summary['result'] = # model.predict() # 'OK'                                            ## Mandatory
summary['score'] = # model.predict_proba() # 0.98                                       ## Mandatory
summary['note'] = # Score는 모델의 추론 예측 결과에 대한 확률 값을 나타냅니다.            ## Mandatory
summary['probability'] = # model.predict_proba() # {'OK': 0.65, 'NG1':0.25, 'NG2':0.1}  ## Optional

self.asset.save_summary(result=summary['result'], score=summary['score'], note=summary['note'], probability=summary['probability'])
```

In [ ]:
## history table 을 시각적 표현을 위해 
%pip install pandas

In [1]:
from src.alo import ALO
import os 
from pprint import pprint
import pandas as pd

alo = ALO()

[INFO][PROCESS][2024-03-11 11:22:04,980]: alolib already exists in local path.
[INFO][PROCESS][2024-03-11 11:22:06,227]: Success installing alolib requirements.txt
[INFO][PROCESS][2024-03-11 11:22:06,231]: Requirement already satisfied: pyyaml==6.0.1 in /home/sehyun.song/miniforge3/envs/tcr/lib/python3.10/site-packages (from -r /home/sehyun.song/Project/alo/dev-240228/alolib/requirements.txt (line 1)) (6.0.1)

[INFO][PROCESS][2024-03-11 11:22:06,234]: Successfully loaded << experimental_plan.yaml >> (file: /home/sehyun.song/Project/alo/dev-240228/solution/experimental_plan.yaml)
[INFO][META][2024-03-11 11:22:06,255]: Loaded solution_metadata: 
None

[INFO][PROCESS][2024-03-11 11:22:06,257]: Process start-time: 240311_112206
[INFO][META][2024-03-11 11:22:06,259]: ALO version = develop
[INFO][PROCESS][2024-03-11 11:22:06,260]: ==================== Start ALO preset ==================== 


### Step 1. 학습 과정을 실행
- setup() : step 별 code 를 git repository 에서 clone 하고, python package 를 설치 합니다. 
- load() : external_data 를 ALO 환경으로 load 합니다. 
- run() : step 을 pipeline 순서대로 실행 합니다. 
- save() : 학습 및 추론 결과를 *_artifact 에 저장 합니다. 실험 내용을 history 에 backup 합니다. 


history 폴더에 아래와 같은 내용들이 저장 됩니다. 


```bash
├── history 
     ├── train
     │   ├── {UTC}-{random}-{contents_name}
     │        ├── experimental_plan.yaml
     │        ├── register_source    ## train docker container 준비물
     │            ├── assets
     │            ├── solution
     │            ├── src
     │            ├── alolib
     │        ├── models
     │        ├── score
     │        ├── report
     │        ├── output
     │        ├── log
     │            ├── experimental_history.json  ## 학습 데이터, 코드, 파라미터 변경 사항 기록
     │
     ├── inference
     │   ├── {UTC}-{random}-{contents_name}
     │        ├── experimental_plan.yaml
     │        ├── register_source    ## inference docker container 준비물
     │            ├── assets
     │            ├── solution
     │            ├── src
     │            ├── alolib
     │        ├── score
     │        ├── extra_output
     │        ├── output
     │        ├── log
     │            ├── experimental_history.json  ## 추론 데이터, 코드, 파라미터 변경 사항 기록
 ```

In [ ]:
## 같 단계별 1회 시작은 필수
train_pipeline = alo.pipeline(pipeline_type='train_pipeline')
train_pipeline.setup()
train_pipeline.load()
train_pipeline.run()
train_pipeline.save()

실험 결과를 확인 합니다.    
- "data_id"    : 실험에 사용된 데이터가 일부라도 변경될 경우 ID 변경. 동일 데이터로 실험 결과 비교 목적     
- "code_id"    : step 코드가 일부라도 변경됨을 감지(간단 프린트 문 추가 만으로도 ID 변경). 동일 코드로 실험 결과 비교에 집중    
- "param_id"   : 파라미터가 추가 되거나 값이 변경될 경우 ID 변경     

In [ ]:
table_list = train_pipeline.history(
     parameter_steps=['input', 'train'])
display(pd.DataFrame(table_list).head(20))

### Step 2. 데이터 변경하여 실험하기

In [ ]:
new_data = './solution/sample_data2/train_data/'  ## 실험에 필요한 데이터로 변경 !!

train_pipeline.load(data_path=new_data)
train_pipeline.run()
train_pipeline.save()

In [ ]:
table_list = train_pipeline.history()
display(pd.DataFrame(table_list).head(20))

### Step 3. 코드 변경하여 실행하기

In [ ]:
train_pipeline.run()
train_pipeline.save()

In [ ]:
table_list = train_pipeline.history()
display(pd.DataFrame(table_list).head(20))

### Step 4. 파라미터 변경하여 실행하기

In [ ]:
train_pipeline.get_parameter('train')['n_estimators'] = 150
train_pipeline.get_parameter('train')
train_pipeline.run()
train_pipeline.save()

### Step 4-2: 파라미터 변경 & 특정 Step 만 반복 실행하기 

In [ ]:
for value in [100, 110, 120, 130]:
    train_pipeline.get_parameter('train')['n_estimators'] = value
    train_pipeline.run(steps = ['train'])
    train_pipeline.save()

In [ ]:
table_list = train_pipeline.history(parameter_steps=['train'])
display(pd.DataFrame(table_list).head(20))

### Step 5: 추론 결과를 확인 (데이터, 코드, 파라미터 변경 가)

In [ ]:
train_id = '20240308T131729Z-91458146-demo-titanic'
train_id = ''
inference_pipeline = alo.pipeline(pipeline_type='inference_pipeline', train_id=train_id)

In [ ]:
inference_pipeline.load()
inference_pipeline.system_envs['inference_history']


In [ ]:
inference_pipeline.setup()
inference_pipeline.load()
inference_pipeline.run()
inference_pipeline.save()

In [ ]:
table_list = inference_pipeline.history()
display(pd.DataFrame(table_list).head(20))

### 실험 결과 등록 (train_id, inference_id 입력)

In [2]:
import getpass

username = input('Username: ')
password = getpass.getpass('Password: ')

print("Your ID : ", username)
print("Your PW : ", password.replace(password, '*' * len(password)))

Your ID :  cism-mgr
Your PW :  ************


In [3]:
infra_setup = "./setting/example_infra_config/infra_config.localtest.yaml"
solution_info ={
    'inference_only': False, # True, False
    'solution_update': False,
    'solution_name': 'titanic-exp-test1',
    'solution_type': 'private',
    'contents_type': {
            'support_labeling': False,
            'inference_result_datatype': 'table', # 'image'
            'train_datatype': 'table', # 'image'
            'labeling_column_name': ''
    },
    'train_gpu': False, ## cpu, gpu
    'inference_gpu': False,
    "inference_arm": False,  # amd, arm  
}


history table 에서 ID 를 확인하고, 입력하여 솔루션을 등록 한다. 
- 학습 실험 과 추론 실험 을 혼합하여 솔루션 등록이 가능 
- train_id, inference_id 가 None 인 경우, 마지막 실험 결과를 등록 
    - history 의 experimental_plan.yaml 를 참조하여 1회 실행하고, 이를 솔루션 등록 함 
    - history 가 오염 될 수 있음을 고려 

In [4]:
train_id = '20240310T065814Z-16031242-demo-titanic'
register = alo.register(
    infra_setup=infra_setup,
    solution_info=solution_info,
    train_id=train_id,   ## history 에서 확인 한 Train ID 입력 하세요.!!
    inference_id='',  ## history 에서 확인 한 Inference ID 를 입력 하세요 !!
)

register.login(username, password)

[INFO][PROCESS][2024-03-11 11:23:04,552]: Successfully loaded << experimental_plan.yaml >> (file: /home/sehyun.song/Project/alo/dev-240228/solution/experimental_plan.yaml)
[INFO][PROCESS][2024-03-11 11:23:04,580]: Successfully loaded << experimental_plan.yaml >> (file: {'asset_source': [{'train_pipeline': [{'source': {'branch': 'release_1.0', 'code': 'http://mod.lge.com/hub/dxadvtech/assets/alo-guide-input.git', 'requirements': ['pandas==1.5.3']}, 'step': 'input'}, {'source': {'branch': 'release_1.0', 'code': 'http://mod.lge.com/hub/dxadvtech/assets/alo-guide-train.git', 'requirements': ['scikit-learn']}, 'step': 'train'}]}], 'control': [{'get_asset_source': 'once'}, {'backup_artifacts': True}, {'backup_log': True}, {'backup_size': 1000}, {'interface_mode': 'memory'}], 'external_path': [{'load_train_data_path': './solution/sample_data/train_data/'}, {'load_inference_data_path': './solution/sample_data/inference_data/'}, {'save_train_artifacts_path': None}, {'save_inference_artifacts_pa

Folder '/home/sehyun.song/Project/alo/dev-240228/.package_list/train_pipeline/' has been removed.
Folder '/home/sehyun.song/Project/alo/dev-240228/.package_list/train_pipeline/' has been created.


[INFO][ASSET][2024-03-11 11:23:05,217][train_pipeline][train]: 

=========================================================== ASSET START ===========================================================
- time (UTC)        : 2024-03-11 02:23:05
- current step      : train
- asset branch.     : release_1.0
- alolib ver.       : develop
- alo ver.          : develop
- load config. keys : dict_keys(['meta', 'x_columns', 'y_column'])
- load data keys    : dict_keys(['dataframe0'])
- load args.        : {'n_estimators': 100}


[INFO][ASSET][2024-03-11 11:23:05,384][train_pipeline][train]: Successfully got model path for saving or loading your AI model: 
 /home/sehyun.song/Project/alo/dev-240228/train_artifacts/models/train/
[WARNING][ASSET][2024-03-11 11:23:05,491][train_pipeline][train]: Please enter the << external_path - save_train_artifacts_path >> in the experimental_plan.yaml.
[INFO][ASSET][2024-03-11 11:23:05,496][train_pipeline][train]: Successfully saved inference summary yaml. 
 >> /hom

({'name': 'demo-titanic', 'version': '2.3.0', 'external_path': [{'load_train_data_path': './solution/sample_data/train_data/'}, {'load_inference_data_path': './solution/sample_data/inference_data/'}, {'save_train_artifacts_path': None}, {'save_inference_artifacts_path': None}, {'load_model_path': None}], 'external_path_permission': [{'aws_key_profile': None}], 'user_parameters': [{'train_pipeline': [{'step': 'input', 'args': [{'input_data_folder': 'train_data', 'x_columns': ['Pclass', 'Sex', 'SibSp', 'Parch'], 'y_column': 'Survived'}], 'ui_args': ['x_columns']}, {'step': 'train', 'args': [{'n_estimators': 100}]}]}, {'inference_pipeline': [{'step': 'input', 'args': [{'input_data_folder': 'inference_data', 'x_columns': ['Pclass', 'Sex', 'SibSp', 'Parch'], 'y_column': None}], 'ui_args': ['x_columns']}, {'step': 'inference', 'args': None}, {'step': 'output', 'args': None}]}], 'asset_source': [{'train_pipeline': [{'step': 'input', 'source': {'code': 'http://mod.lge.com/hub/dxadvtech/assets/

AttributeError: 'tuple' object has no attribute 'items'

In [ ]:
register.debugging = True  ## default: False (skip 항목: docker 생성, solution 등록)
register.run(username=username, password=password)

In [ ]:
register.exp_yaml

In [ ]:
register.run_train(
    status_period = 10, ## 몇 초 간격으로 학습 상태를 체크할 것인지 설정
    delete_instance = False,   ## 학습 테스트에 사용한 solution instance & stream 을 삭제할 것인지 설정 
    delete_solution= False,  ## solution 을 삭제할 것인지 설정
    )